In [ ]:
# !pip install tfx

In [ ]:
# import library
import os
import sys
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/DICODING/MLOPS/proyek_akhir

/content/drive/MyDrive/DICODING/MLOPS/proyek_akhir


In [ ]:
PIPELINE_NAME = "twnklstar-pipeline"

# Pipeline inputs
DATA_ROOT = "/content/drive/MyDrive/DICODING/MLOPS/proyek_akhir/data"
TRANSFORM_MODULE_FILE = "/content/drive/MyDrive/DICODING/MLOPS/proyek_akhir/modules/review_transform.py"
TRAINER_MODULE_FILE = "/content/drive/MyDrive/DICODING/MLOPS/proyek_akhir/modules/review_trainer.py"

# Pipeline outputs
OUTPUT_BASE = "/content/output"
serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [ ]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "--direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args
    )

In [ ]:
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: /content/output/twnklstar-pipeline
INFO:absl:Generating ephemeral wheel package for '/content/drive/MyDrive/DICODING/MLOPS/proyek_akhir/modules/review_transform.py' (including modules: ['review_trainer', 'review_transform', 'components']).
INFO:absl:User module package has hash fingerprint version 775932b761375edb024b418a806d2ee03b5ce454964037cb265b2ded1c5ed30a.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpdpdk1kvc/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp6dw3gnzc', '--dist-dir', '/tmp/tmplm633u3s']
INFO:absl:Successfully built user code wheel distribution at '/content/output/twnklstar-pipeline/_wheels/tfx_user_code_Transform-0.0+775932b761375edb024b418a806d2ee03b5ce454964037cb265b2ded1c5ed30a-py3-none-any.whl'; target user module is

INFO:absl:Node CsvExampleGen depends on [].
INFO:absl:Node CsvExampleGen is scheduled.
INFO:absl:Node Latest_blessed_model_resolver depends on [].
INFO:absl:Node Latest_blessed_model_resolver is scheduled.
INFO:absl:Node StatisticsGen depends on ['Run[CsvExampleGen]'].
INFO:absl:Node StatisticsGen is scheduled.
INFO:absl:Node SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Node SchemaGen is scheduled.
INFO:absl:Node ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Node ExampleValidator is scheduled.
INFO:absl:Node Transform depends on ['Run[CsvExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Node Transform is scheduled.
INFO:absl:Node Trainer depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Node Trainer is scheduled.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088  


Epoch 1: val_binary_accuracy improved from -inf to 0.81118, saving model to /content/output/twnklstar-pipeline/Trainer/model/10/Format-Serving


INFO:absl:Function `_wrapped_model` contains input name(s) table_handle, 52102, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_none_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_52102, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/output/twnklstar-pipeline/Trainer/model/10/Format-Serving/fingerprint.pb


1000/1000 [==============================] - 25s 23ms/step - loss: 0.3054 - binary_accuracy: 0.8625 - val_loss: 0.6445 - val_binary_accuracy: 0.8112
Epoch 2/10
 998/1000 [============================>.] - ETA: 0s - loss: 0.1275 - binary_accuracy: 0.9423

1000/1000 [==============================] - 15s 15ms/step - loss: 0.1273 - binary_accuracy: 0.9424
Epoch 3/10
 998/1000 [============================>.] - ETA: 0s - loss: 0.0578 - binary_accuracy: 0.9775

1000/1000 [==============================] - 15s 15ms/step - loss: 0.0577 - binary_accuracy: 0.9775
Epoch 4/10
 126/1000 [==>...........................] - ETA: 10s - loss: 0.0439 - binary_accuracy: 0.9815

1000/1000 [==============================] - 2s 2ms/step - loss: 0.0438 - binary_accuracy: 0.9815


INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sentiment has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) table_handle, 112877, resource with unsupported characters which will be renamed to model_text_vectorization_string_lookup_hash_table_lookup_lookuptablefindv2_table_handle, model_embedding_embedding_lookup_112877, model_dense_2_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/output/twnklstar-pipeline/Trainer/model/10/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to /content/output/twnklstar-pipeline/Trainer/model/10/Format-Serving. ModelRun written to /content/output/twnklstar-pipeline/Trainer/model_run/10
INFO:absl:Cleaning up stateless exec

In [ ]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/twnklstar-pipeline/ (stored 0%)
  adding: content/output/twnklstar-pipeline/_wheels/ (stored 0%)
  adding: content/output/twnklstar-pipeline/_wheels/tfx_user_code_Transform-0.0+a1551ae1aa7d71a0c8f256e9d0be7f78719c633ffe02b5f7c42050195973a59e-py3-none-any.whl (deflated 18%)
  adding: content/output/twnklstar-pipeline/_wheels/tfx_user_code_Trainer-0.0+775932b761375edb024b418a806d2ee03b5ce454964037cb265b2ded1c5ed30a-py3-none-any.whl (deflated 18%)
  adding: content/output/twnklstar-pipeline/_wheels/tfx_user_code_Transform-0.0+775932b761375edb024b418a806d2ee03b5ce454964037cb265b2ded1c5ed30a-py3-none-any.whl (deflated 19%)
  adding: content/output/twnklstar-pipeline/_wheels/tfx_user_code_Trainer-0.0+a1551ae1aa7d71a0c8f256e9d0be7f78719c633ffe02b5f7c42050195973a59e-py3-none-any.whl (deflated 18%)
  adding: content/output/twnklstar-pipeline/CsvExampleGen/ (stored 0%)
  adding: content/output/twnklstar-pipeline/CsvExampleGen/example